#### TODO 

Insert better comments, maybe a plot or two
- grab loss from train, and then plot it...
- give average time per batch, and min time per batch from train?

Then, experiment a bit with 4 or 6 mpi tasks, 16 or 24 layers, 1 backward skip down, and 2 forward, speedup?
- play with anything else, like num channels?

#### Apply layer-parallel Torchbraid to simple MNIST problem (fashion or digits)
- See `start0_install_mpi_notebook`, and `start1_simple_mpi_notebook` for setting up MPI-compatible Jupyter installation

In [2]:
# Connect to local ipython cluster.  Note, the ipcluster profile name must match the
# below named profile. Here, we use 'mpi', but you can name the cluster profile anything
from ipyparallel import Client, error
cluster = Client(profile='mpi')
print('profile:', cluster.profile)
print("IDs:", cluster.ids) 

profile: mpi
IDs: [0, 1]


In [3]:
%%px
# You must upate the sys.path to point to your Torchbraid location
from __future__ import print_function

import statistics as stats
import sys
from timeit import default_timer as timer

import matplotlib.pyplot as pyplot
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from mpi4py import MPI

import sys 
sys.path.append("/Users/jacobschroder/joint_repos/torchbraid/torchbraid_py3_10/torchbraid")
sys.path.append("/Users/jacobschroder/joint_repos/torchbraid/torchbraid_py3_10")
import torchbraid
import torchbraid.utils

In [4]:
# Download the data
# Depending on parallel setting, may want to do in parallel with `%%px` command
from torchvision import datasets, transforms
datasets.MNIST('./digit-data', download=True)
datasets.FashionMNIST('./fashion-data', download=True)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./fashion-data
    Split: Train

In [5]:
%%px
# Network architecture is Open + ResNet + Close
# The StepLayer defines the ResNet (ODENet) 

class OpenFlatLayer(nn.Module):
  ''' 
  Opening layer has no parameters, replicates image number of channels times
  '''
  def __init__(self, channels):
    super(OpenFlatLayer, self).__init__()
    self.channels = channels

  def forward(self, x):
    s = len(x.shape) * [1]
    s[1] = self.channels
    x = x.repeat(s)
    return x

class CloseLayer(nn.Module):
  '''
  Dense closing classification layer
  '''
  def __init__(self, channels):
    super(CloseLayer, self).__init__()
    self.fc1 = nn.Linear(channels * 28 * 28, 32)
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)


class StepLayer(nn.Module):
  '''
  ResNet composed of convolutional layers
  '''
  def __init__(self, channels):
    super(StepLayer, self).__init__()
    ker_width = 3
    self.conv1 = nn.Conv2d(channels, channels, ker_width, padding=1)
    self.conv2 = nn.Conv2d(channels, channels, ker_width, padding=1)

  def forward(self, x):
    return F.relu(self.conv2(F.relu(self.conv1(x))))

In [6]:
%%px
# Serial network class
class SerialNet(nn.Module):
  def __init__(self, channels=12, local_steps=8, Tf=1.0, 
               serial_nn=None, open_nn=None, close_nn=None):
    super(SerialNet, self).__init__()

    if open_nn is None:
      self.open_nn = OpenFlatLayer(channels)
    else:
      self.open_nn = open_nn

    if serial_nn is None:
      step_layer = lambda: StepLayer(channels)
      numprocs = 1
      parallel_nn = torchbraid.LayerParallel(MPI.COMM_SELF, step_layer, numprocs*local_steps, Tf,
                                             max_fwd_levels=1, max_bwd_levels=1, max_iters=1)
      parallel_nn.setPrintLevel(0, True)
      self.serial_nn = parallel_nn.buildSequentialOnRoot()
    else:
      self.serial_nn = serial_nn

    if close_nn is None:
      self.close_nn = CloseLayer(channels)
    else:
      self.close_nn = close_nn

  def forward(self, x):
    x = self.open_nn(x)
    x = self.serial_nn(x)
    x = self.close_nn(x)
    return x

In [7]:
%%px
# Parallel network class
# local_steps: number of ResNet layers per processor
# all other parameter definitions are in the params['...'] block
class ParallelNet(nn.Module):
  def __init__(self, channels=12, local_steps=8, Tf=1.0, max_levels=1, max_iters=1, 
               fwd_max_iters=0, print_level=0, braid_print_level=0, cfactor=4, 
               fine_fcf=False, skip_downcycle=True, fmg=False, relax_only_cg=0,
               user_mpi_buf=False, gpu_direct_commu=False):
    super(ParallelNet, self).__init__()

    step_layer = lambda: StepLayer(channels)
    numprocs = MPI.COMM_WORLD.Get_size()
    
    self.parallel_nn = torchbraid.LayerParallel(MPI.COMM_WORLD, step_layer, local_steps*numprocs, Tf,
                                                max_fwd_levels=max_levels, max_bwd_levels=max_levels,
                                                max_iters=max_iters, user_mpi_buf=user_mpi_buf,
                                                gpu_direct_commu=gpu_direct_commu)
    if fwd_max_iters > 0:
      print('fwd_amx_iters', fwd_max_iters)
      self.parallel_nn.setFwdMaxIters(fwd_max_iters)
        
    self.parallel_nn.setPrintLevel(print_level, True)
    self.parallel_nn.setPrintLevel(braid_print_level, False)
    self.parallel_nn.setCFactor(cfactor)
    self.parallel_nn.setSkipDowncycle(skip_downcycle)
    self.parallel_nn.setBwdRelaxOnlyCG(relax_only_cg)
    self.parallel_nn.setFwdRelaxOnlyCG(relax_only_cg)
    if fmg:
      self.parallel_nn.setFMG()
    
    self.parallel_nn.setNumRelax(1)  # FCF relaxation default on coarse levels
    if not fine_fcf:
      self.parallel_nn.setNumRelax(0, level=0)  # Set F-Relaxation only on the fine grid
    else:
      self.parallel_nn.setNumRelax(1, level=0)  # Set FCF-Relaxation on the fine grid

    # this object ensures that only the LayerParallel code runs on ranks!=0
    compose = self.compose = self.parallel_nn.comp_op()

    # by passing this through 'compose' (mean composition: e.g. OpenFlatLayer o channels)
    # on processors not equal to 0, these will be None (there are no parameters to train there)
    self.open_nn = compose(OpenFlatLayer, channels)
    self.close_nn = compose(CloseLayer, channels)

  def saveSerialNet(self, name):
    serial_nn = self.parallel_nn.buildSequentialOnRoot()
    if MPI.COMM_WORLD.Get_rank() == 0:
      s_net = SerialNet(-1, -1, -1, serial_nn=serial_nn, open_nn=self.open_nn, close_nn=self.close_nn)
      s_net.eval()
      torch.save(s_net, name)

  def getDiagnostics(self):
    return self.parallel_nn.getDiagnostics()

  def forward(self, x):
    # by passing this through 'o' (mean composition: e.g. self.open_nn o x)
    # this makes sure this is run on only processor 0

    x = self.compose(self.open_nn, x)
    x = self.parallel_nn(x)
    x = self.compose(self.close_nn, x)

    return x

In [8]:
%%px
# Train model for one epoch
def train(rank, params, model, train_loader, optimizer, epoch, compose, device):
  model.train()
  criterion = nn.CrossEntropyLoss()
  total_time = 0.0
  for batch_idx, (data, target) in enumerate(train_loader):
    start_time = timer()
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = compose(criterion, output, target)
    loss.backward()
    stop_time = timer()
    optimizer.step()

    total_time += stop_time - start_time
    if batch_idx % params['log_interval'] == 0:
      root_print(rank, 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime Per Batch {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
               100. * batch_idx / len(train_loader), loss.item(), total_time / (batch_idx + 1.0)))

  root_print(rank, 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime Per Batch {:.6f}'.format(
    epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
           100. * (batch_idx + 1) / len(train_loader), loss.item(), total_time / (batch_idx + 1.0)))

In [9]:
%%px
# Evaluate model on validation data
def test(rank, model, test_loader, compose, device):
  model.eval()
  test_loss = 0
  correct = 0
  criterion = nn.CrossEntropyLoss()
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += compose(criterion, output, target).item()

      if rank == 0:
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  root_print(rank, '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [10]:
%%px
# Parallel printing helper function
def root_print(rank, s):
  if rank == 0:
    print(s)

# Compute number of parallel-in-time multigrid levels
def compute_levels(num_steps, min_coarse_size, cfactor):
  from math import log, floor

  # Find L such that ( max_L min_coarse_size*cfactor**L <= num_steps)
  levels = floor(log(float(num_steps) / min_coarse_size, cfactor)) + 1

  if levels < 1:
    levels = 1
  return levels

In [11]:
%%px
#Set parameters for network architecture and layer-parallel 
params = {}
params['seed'] = 1           # random seed
params['log_interval'] = 10  # how many batches to wait before logging training status
params['dataset'] = 'digits' # 'digits' or 'fashion' MNIST 
#
params['steps'] = 16         # number of times steps in the resnet layer-parallel part
params['channels'] = 4       # number of channels in resnet layer
params['Tf'] = 1.0           # final time for resnet layer-parallel part
#
params['percent_data'] = 0.1 # how much of the data to read in and use for training/testing
params['batch_size'] = 50    # input batch size for training
params['epochs'] = 2         # number of epochs to train
params['lr'] = 0.01          # learning rate
#
params['lp_max_levels'] = 3         # layer parallel max number levels 
params['lp_max_iters'] = 2          # layer parallel max iterations
params['lp_fwd_max_iters'] = -1     # layer parallel max (forward) iterations, if -1 use lp_max_iters
params['lp_print_level'] = 0        # layer parallel internal print level: 0, 1, 2, 3 
params['lp_braid_print_level'] = 0  # layer parallel braid print level: 0, 1, 2, 3 
params['lp_cfactor'] = 4            # layer parallel coarsening factor
params['lp_fine_fcf'] = False       # layer parallel fine FCF on or off 
params['no_cuda'] = False           # disables CUDA training
params['warm_up'] = False           # warm up for GPU timings
params['lp_user_mpi_buf'] = False   # layer parallel use user-defined mpi buffers 
params['lp_use_downcycle']= False   # layer parallel use downcycle on or off
params['lp_gpu_direct_commu'] = False # layer parallel GPU direct communication

In [12]:
%%px
# Begin setting up run-time environment 
# MPI information
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
procs = comm.Get_size()

# Use device or CPU?
use_cuda = not params['no_cuda'] and torch.cuda.is_available()
device, host = torchbraid.utils.getDevice(comm=comm)
if not use_cuda:
  device = torch.device("cuda" if use_cuda else "cpu")
print(f'Run info rank: {rank}: | Device: {device} | Host: {host}')

# Set seed for reproducibility
torch.manual_seed(params['seed'])

# Compute max number of layer-parallel levels
if params['lp_max_levels'] == -1:
  min_coarse_size = 3
  params['lp_max_levels'] = compute_levels(params['steps'], min_coarse_size, params['lp_cfactor'])

# Compute number of steps in ResNet per processor
local_steps = int(params['steps'] / procs)
if params['steps'] % procs != 0:
  root_print(rank, 'Steps must be an even multiple of the number of processors: %d %d' % (params['steps'], procs))


[stdout:0] No GPUs to be used, CPU only
Run info rank: 0: | Device: cpu | Host: cpu


[stdout:1] Run info rank: 1: | Device: cpu | Host: cpu


In [13]:
%%px
# read in Digits MNIST or Fashion MNIST
if params['dataset'] == 'digits':
  root_print(rank, '-- Using Digit MNIST')
  transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                                  ])
  dataset = datasets.MNIST('./digit-data', download=False, transform=transform)
else:
  root_print(rank, '-- Using Fashion MNIST')
  transform = transforms.Compose([transforms.ToTensor()])
  dataset = datasets.FashionMNIST('./fashion-data', download=False, transform=transform)

train_size = int(50000 * params['percent_data'])
test_size = int(10000 * params['percent_data'])
train_set = torch.utils.data.Subset(dataset, range(train_size))
test_set = torch.utils.data.Subset(dataset, range(train_size, train_size + test_size))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=params['batch_size'], shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=params['batch_size'], shuffle=False)

[stdout:0] -- Using Digit MNIST


In [19]:
%%px
# Create layer-parallel network
# Note this can be done on only one processor, but will be slow
model = ParallelNet(channels=params['channels'],
                    local_steps=local_steps,
                    max_levels=params['lp_max_levels'],
                    max_iters=params['lp_max_iters'],
                    fwd_max_iters=params['lp_fwd_max_iters'],
                    print_level=params['lp_print_level'],
                    braid_print_level=params['lp_braid_print_level'],
                    cfactor=params['lp_cfactor'],
                    fine_fcf=params['lp_fine_fcf'],
                    skip_downcycle=not params['lp_use_downcycle'],
                    fmg=False, 
                    Tf=params['Tf'],
                    relax_only_cg=False,
                    user_mpi_buf=params['lp_user_mpi_buf'],
                    gpu_direct_commu=params['lp_gpu_direct_commu']).to(device)

compose = model.compose
model.parallel_nn.fwd_app.setTimerFile(
  'b_fwd_s_%d_c_%d_bs_%d_p_%d_gpuc_%d'%(params['steps'], params['channels'], params['batch_size'], procs, params['lp_gpu_direct_commu']) )
model.parallel_nn.bwd_app.setTimerFile( 
  'b_bwd_s_%d_c_%d_bs_%d_p_%d_gpuc_%d'%(params['steps'], params['channels'], params['batch_size'], procs, params['lp_gpu_direct_commu']) )   

In [20]:
%%px
# Declare optimizer
optimizer = optim.SGD(model.parameters(), lr=params['lr'], momentum=0.9)

# For better timings (especially with GPUs) do a little warm up
if params['warm_up']:
  warm_up_timer = timer()
  train(rank=rank, args=params, model=model, train_loader=train_loader, optimizer=optimizer, epoch=0,
        compose=compose, device=device)
  if force_lp:
    model.parallel_nn.timer_manager.resetTimers()
    model.parallel_nn.fwd_app.resetBraidTimer()
    model.parallel_nn.bwd_app.resetBraidTimer()
  if use_cuda:
    torch.cuda.synchronize()
  root_print(rank, f'Warm up timer {timer() - warm_up_timer}')

In [21]:
%%px
# Carry out parallel training
epoch_times = []
test_times = []

for epoch in range(1, params['epochs'] + 1):
  start_time = timer()
  train(rank=rank, params=params, model=model, train_loader=train_loader, optimizer=optimizer, epoch=epoch,
        compose=compose, device=device)
  epoch_times += [timer() - start_time]

  start_time = timer()
  test(rank=rank, model=model, test_loader=test_loader, compose=compose, device=device)
  test_times += [timer() - start_time]

# Print out Braid internal timings, if desired
#timer_str = model.parallel_nn.getTimersString()
#root_print(rank, timer_str)

root_print(rank,
           f'TIME PER EPOCH: {"{:.2f}".format(stats.mean(epoch_times))} '
           f'{("(1 std dev " + "{:.2f}".format(stats.mean(epoch_times))) if len(epoch_times) > 1 else ""}')
root_print(rank,
           f'TIME PER TEST:  {"{:.2f}".format(stats.mean(test_times))} '
           f'{("(1 std dev " + "{:.2f}".format(stats.mean(test_times))) if len(test_times) > 1 else ""}')

[stdout:0] Train Epoch: 1 [0/5000 (0%)]	Loss: 2.308671	Time Per Batch 0.395477
Train Epoch: 1 [500/5000 (10%)]	Loss: 1.484470	Time Per Batch 0.393349
Train Epoch: 1 [1000/5000 (20%)]	Loss: 0.762372	Time Per Batch 0.387455
Train Epoch: 1 [1500/5000 (30%)]	Loss: 0.475898	Time Per Batch 0.388980
Train Epoch: 1 [2000/5000 (40%)]	Loss: 0.657345	Time Per Batch 0.387190
Train Epoch: 1 [2500/5000 (50%)]	Loss: 0.107705	Time Per Batch 0.385901
Train Epoch: 1 [3000/5000 (60%)]	Loss: 0.462533	Time Per Batch 0.384882
Train Epoch: 1 [3500/5000 (70%)]	Loss: 0.388873	Time Per Batch 0.383329
Train Epoch: 1 [4000/5000 (80%)]	Loss: 0.242518	Time Per Batch 0.382303
Train Epoch: 1 [4500/5000 (90%)]	Loss: 0.355126	Time Per Batch 0.382884
Train Epoch: 1 [5000/5000 (100%)]	Loss: 0.518656	Time Per Batch 0.382135

Test set: Average loss: 0.0089, Accuracy: 864/1000 (86%)

Train Epoch: 2 [0/5000 (0%)]	Loss: 0.321612	Time Per Batch 0.373599
Train Epoch: 2 [500/5000 (10%)]	Loss: 0.353730	Time Per Batch 0.374180
Tra

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]

In [18]:
%%px
# Create Serial network, if desired, for comparison
serial_model = SerialNet(channels=params['channels'], local_steps=local_steps, 
                         Tf=params['Tf']).to(device)
serial_compose = lambda op, *p: op(*p)

# Declare optimizer
optimizer = optim.SGD(serial_model.parameters(), lr=params['lr'], momentum=0.9)
epoch_times = []
test_times = []

# Begin training
for epoch in range(1, params['epochs'] + 1):
  start_time = timer()
  train(rank=rank, params=params, model=serial_model, train_loader=train_loader, 
        optimizer=optimizer, epoch=epoch,compose=serial_compose, device=device)
  epoch_times += [timer() - start_time]

  start_time = timer()
  test(rank=rank, model=serial_model, test_loader=test_loader, 
       compose=serial_compose, device=device)
  test_times += [timer() - start_time]

root_print(rank,
           f'TIME PER EPOCH: {"{:.2f}".format(stats.mean(epoch_times))} '
           f'{("(1 std dev " + "{:.2f}".format(stats.mean(epoch_times))) if len(epoch_times) > 1 else ""}')
root_print(rank,
           f'TIME PER TEST:  {"{:.2f}".format(stats.mean(test_times))} '
           f'{("(1 std dev " + "{:.2f}".format(stats.mean(test_times))) if len(test_times) > 1 else ""}')

[stdout:0] Train Epoch: 1 [0/5000 (0%)]	Loss: 2.301360	Time Per Batch 0.086586
Train Epoch: 1 [500/5000 (10%)]	Loss: 1.357578	Time Per Batch 0.089633
Train Epoch: 1 [1000/5000 (20%)]	Loss: 0.752395	Time Per Batch 0.089501
Train Epoch: 1 [1500/5000 (30%)]	Loss: 0.560271	Time Per Batch 0.088574
Train Epoch: 1 [2000/5000 (40%)]	Loss: 0.592479	Time Per Batch 0.087377
Train Epoch: 1 [2500/5000 (50%)]	Loss: 0.144857	Time Per Batch 0.087100
Train Epoch: 1 [3000/5000 (60%)]	Loss: 0.467642	Time Per Batch 0.086324
Train Epoch: 1 [3500/5000 (70%)]	Loss: 0.498877	Time Per Batch 0.085941
Train Epoch: 1 [4000/5000 (80%)]	Loss: 0.268307	Time Per Batch 0.085771
Train Epoch: 1 [4500/5000 (90%)]	Loss: 0.349492	Time Per Batch 0.086924
Train Epoch: 1 [5000/5000 (100%)]	Loss: 0.563418	Time Per Batch 0.087425

Test set: Average loss: 0.0092, Accuracy: 858/1000 (86%)

Train Epoch: 2 [0/5000 (0%)]	Loss: 0.346644	Time Per Batch 0.087519
Train Epoch: 2 [500/5000 (10%)]	Loss: 0.452453	Time Per Batch 0.082889
Tra

%px:   0%|          | 0/2 [00:00<?, ?tasks/s]